In [35]:
print("hello world")

hello world


In [58]:
# get some libraries that will be useful
import re
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.image as mpimg
import stop_words
from sklearn.ensemble import AdaBoostClassifier

# the Naive Bayes model
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
# function to split the data for cross-validation
from sklearn.model_selection import train_test_split
# function for transforming documents into counts
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# function for encoding categories
from sklearn.preprocessing import LabelEncoder

# Outputs the headlines dataset.
# X is a list of headlines
# y is a list of binary labels, 0 stands for Haaretz and 1 for Israel Hayom
def load_dataset(filenames=['haaretz.csv','israelhayom.csv']):
    cur_y = 0
    X = pd.DataFrame()
    y = np.empty(0,dtype=np.int32)
    for filename in filenames:
        train_cur = pd.read_csv(filename, header=None)
        X = pd.concat([X,train_cur[0]])
        y = np.append(y,cur_y*np.ones(len(train_cur),dtype=np.int32))
        cur_y += 1
    X = [x[0] for x in X.values.tolist()]
    y = y.tolist()
    return X,y

# grab the data
X,Y = load_dataset()


In [59]:
def normalize_text(s):
    s = s.lower()
    
    # remove punctuation that is not word-internal (e.g., hyphens, apostrophes)
    #s = re.sub('\s\W',' ',s)
   # s = re.sub('\W\s',' ',s)
    
    # make sure we didn't introduce any double spaces
    s = re.sub('\s+',' ',s)
    
    return s

In [60]:
X = [normalize_text(s) for s in X]

In [73]:
stop_words_list=stop_words.get_stop_words('english')

# pull the data into vectors
vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1,6), analyzer='char_wb', lowercase=False, strip_accents='unicode', stop_words=stop_words_list)
x = vectorizer.fit_transform(X)
encoder = LabelEncoder()
y = encoder.fit_transform(Y)

# split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print("y %", (y_train > 0).sum()/(y_train.shape))
# take a look at the shape of each of these
print(x.shape)
print(x2.shape)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

y % [ 0.58463591]
(6558, 54100)
(6558, 54100)
(5246, 54100)
(5246,)
(1312, 54100)
(1312,)


In [62]:
print(np.array(X).shape)


(6558,)


In [63]:
svm = svm.LinearSVC()
svm.fit(x_train, y_train)
print("svm", nb.score(x_test, y_test))
nb = MultinomialNB()
nb.fit(x_train, y_train)
print("naive baise", nb.score(x_test, y_test))

svm 0.791920731707
naive baise 0.734756097561


In [64]:
try1 = ["Occupation Occupation Occupation", "west bank", "right wing","sarah","Bibi and Yair",
                             "Israeli IDF iran minister"]

test = vectorizer.transform(try1)
res= nb.predict(test)

for i,cur in enumerate(try1):
    tag = "haaretz" if res[i] == 0 else "Israel Hayom"
    print(cur, ": ", tag)


Occupation Occupation Occupation :  haaretz
west bank :  haaretz
right wing :  haaretz
sarah :  Israel Hayom
Bibi and Yair :  Israel Hayom
Israeli IDF iran minister :  Israel Hayom
